In [ ]:
import os
from keras.models import Model
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
import numpy as np

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from livelossplot import PlotLossesKeras


In [ ]:
import keras_cv
from keras_cv import bounding_box
import os
from luketils import visualization

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras

In [ ]:
pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
BATCH_SIZE = 16

train_generator = ImageDataGenerator(rotation_range=90, 
                                     brightness_range=[0.1, 0.7],
                                     width_shift_range=0.5, 
                                     height_shift_range=0.5,
                                     horizontal_flip=True, 
                                     vertical_flip=True,
                                     validation_split=0.15,
                                     preprocessing_function=preprocess_input) # VGG16 preprocessing

test_generator = ImageDataGenerator(preprocessing_function=preprocess_input) # VGG16 preprocessing

In [ ]:
download_dir = Path('/content/drive/MyDrive/VOC2008/')

In [ ]:
download_dir

PosixPath('/content/drive/MyDrive/VOC2008')

In [ ]:
class_subset = sorted(os.listdir(download_dir/'JPEGImages'))[:20]

In [ ]:
class_subset

['2007_001185.jpg',
 '2007_001225.jpg',
 '2007_001239.jpg',
 '2007_001284.jpg',
 '2007_001288.jpg',
 '2007_001289.jpg',
 '2007_001299.jpg',
 '2007_001311.jpg',
 '2007_001321.jpg',
 '2007_001340.jpg',
 '2007_001377.jpg',
 '2007_001397.jpg',
 '2007_001408.jpg',
 '2007_001416.jpg',
 '2007_001420.jpg',
 '2007_001423.jpg',
 '2007_001430.jpg',
 '2007_001439.jpg',
 '2007_001457.jpg',
 '2007_001458.jpg']

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(
    rescale=1/255.,              # normalize pixel values between 0-1
    brightness_range=[0.1, 0.7], # specify the range in which to decrease/increase brightness
    width_shift_range=0.5,       # shift the width of the image 50%
    rotation_range=90,           # random rotation by 90 degrees
    horizontal_flip=True,        # 180 degree flip horizontally
    vertical_flip=True,          # 180 degree flip vertically
    validation_split=0.15        # 15% of the data will be used for validation at end of each epoch
)

In [ ]:
#os.mkdir(download_dir/'food-101')
#os.mkdir(download_dir/'food-101/train')
#os.mkdir(download_dir/'food-101/valid')

train_data_dir = download_dir/'food-101/train'
test_data_dir = download_dir/'food-101/test'
valid_data_dir = download_dir/'food-101/valid'

class_id = [
    "Aeroplane",
    "Bicycle",
    "Bird",
    "Boat",
    "Bottle",
    "Bus",
    "Car",
    "Cat",
    "Chair",
    "Cow",
    "Dining Table",
    "Dog",
    "Horse",
    "Motorbike",
    "Person",
    "Potted Plant",
    "Sheep",
    "Sofa",
    "Train",
    "Tvmonitor",
]

class_subset = sorted(os.listdir(download_dir/'JPEGImages'))[:20] # Using only the first 10 classes

traingen = train_generator.flow_from_directory('/content/drive/MyDrive/VOC2008/JPEGImages',
                                               target_size=(512, 512),
                                               class_mode='categorical',
                                               classes=class_subset,
                                               subset='training',
                                               batch_size=BATCH_SIZE, 
                                               shuffle=True,
                                               seed=42)

validgen = train_generator.flow_from_directory('/content/drive/MyDrive/VOC2008/food-101/train/',
                                               target_size=(512, 512),
                                               class_mode='categorical',
                                               classes=class_subset,
                                               subset='validation',
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               seed=42)

testgen = test_generator.flow_from_directory('//content/drive/MyDrive/VOC2008/food-101/test/',
                                             target_size=(512, 512),
                                             class_mode=None,
                                             classes=class_id,
                                             batch_size=1,
                                             shuffle=False,
                                             seed=42)

Found 0 images belonging to 20 classes.
Found 0 images belonging to 20 classes.
Found 0 images belonging to 20 classes.


Compiles a model integrated with VGG16 pretrained layers
input_shape: tuple - the shape of input images (width, height, channels)
n_classes: int - number of classes for the output layer
optimizer: string - instantiated optimizer to use for training. Defaults to 'RMSProp'
fine_tune: int - The number of pre-trained layers to unfreeze.
If set to 0, all pretrained layers will freeze during training

In [ ]:
def create_model(input_shape, n_classes, optimizer='rmsprop', fine_tune=0):
    # Pretrained convolutional layers are loaded using the Imagenet weights.
    # Include_top is set to False, in order to exclude the model's fully-connected layers.
    conv_base = VGG16(include_top=False,
                     weights='imagenet', 
                     input_shape=input_shape)
    
    # Defines how many layers to freeze during training.
    # Layers in the convolutional base are switched from trainable to non-trainable
    # depending on the size of the fine-tuning parameter.
    if fine_tune > 0:
        for layer in conv_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in conv_base.layers:
            layer.trainable = False

    # Create a new 'top' of the model (i.e. fully-connected layers).
    # This is 'bootstrapping' a new top_model onto the pretrained layers.
    top_model = conv_base.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(4096, activation='relu')(top_model)
    top_model = Dense(1072, activation='relu')(top_model)
    top_model = Dropout(0.2)(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)
    
    # Group the convolutional base and new fully-connected layers into a Model object.
    model = Model(inputs=conv_base.input, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
from traitlets.traitlets import validate
input_shape = (512, 512, 3)
optim_1 = Adam(learning_rate=0.001)
n_classes=20

n_epochs = BATCH_SIZE

# First we'll train the model without Fine-tuning
vgg_model = create_model(input_shape, n_classes, optim_1, fine_tune=0)

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
from livelossplot.inputs.keras import PlotLossesCallback

plot_loss_1 = PlotLossesCallback()

# ModelCheckpoint callback - save best weights
tl_checkpoint_1 = ModelCheckpoint(filepath='tl_model_v1.weights.best.hdf5',
                                  save_best_only=True,
                                  verbose=1)

# EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',
                           patience=10,
                           restore_best_weights=True,
                           mode='min')

In [ ]:
vgg_history = vgg_model.fit(traingen,
                            batch_size=BATCH_SIZE,
                            epochs=EPOCHS,
                            validation_data=validgen,
                            steps_per_epoch=traingen.samples,
                            validation_steps=validgen.samples,
                            callbacks=[tl_checkpoint_1, early_stop, plot_loss_1],
                            verbose=1)

In [ ]:
# Generate predictions
vgg_model.load_weights('tl_model_v1.weights.best.hdf5') # initialize the best trained weights

true_classes = testgen.classes
class_indices = traingen.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())

vgg_preds = vgg_model.predict(testgen)
vgg_pred_classes = np.argmax(vgg_preds, axis=1)

In [ ]:
from sklearn.metrics import accuracy_score

vgg_acc = accuracy_score(true_classes, vgg_pred_classes)
print("VGG16 Model Accuracy without Fine-Tuning: {:.2f}%".format(vgg_acc * 100))

VGG16 Model Accuracy with Fine-Tuning: 88.26%


In [ ]:
# Reset our image data generators
traingen.reset()
validgen.reset()
testgen.reset()

# Use a smaller learning rate
optim_2 = Adam(lr=0.0001)

# Re-compile the model, this time leaving the last 2 layers unfrozen for Fine-Tuning
vgg_model_ft = create_model(input_shape, n_classes, optim_2, fine_tune=2)

In [ ]:
plot_loss_2 = PlotLossesCallback()

# Retrain model with fine-tuning
vgg_ft_history = vgg_model_ft.fit(traingen,
                                  batch_size=BATCH_SIZE,
                                  epochs=n_epochs,
                                  validation_data=validgen,
                                  steps_per_epoch=n_steps, 
                                  validation_steps=n_val_steps,
                                  callbacks=[tl_checkpoint_1, early_stop, plot_loss_2],
                                  verbose=1)


In [ ]:
# Generate predictions
vgg_model_ft.load_weights('tl_model_v1.weights.best.hdf5') # initialize the best trained weights

vgg_preds_ft = vgg_model_ft.predict(testgen)
vgg_pred_classes_ft = np.argmax(vgg_preds_ft, axis=1)

In [ ]:
vgg_acc_ft = accuracy_score(true_classes, vgg_pred_classes_ft)
print("VGG16 Model Accuracy with Fine-Tuning: {:.2f}%".format(vgg_acc_ft * 100))

Found 15400 images belonging to 20 classes.
From Scratch Model Accuracy with Fine-Tuning: 88.26%
